# Module 2.  BYOC(Bring Your Own Container): Multi-Model Endpoint Deployment on MMS(Multi Model Server)
---

본 모듈에서는 멀티모델 엔드포인트로 여러 모델들을 단일 호스팅 인스턴스에 배포합니다. 이를 위해  MMS(Multi Model Server)와 SageMaker Inference Toolkit가 포된 사용자 정의 컨테이너를 빌드합니다. 노트북 실행에는 약 30분 가량 소요됩니다. 사용자 정의 컨테이너를 빌드하여 모델을 배포하는 모범 예시는 아래 웹페이지를 참좨 주세요.
- https://github.com/awslabs/amazon-sagemaker-examples/tree/master/advanced_functionality/multi_model_bring_your_own

### MMS 
2017년 12월 초  MXNet 1.0 릴리스 시 최초 공개된 모델 서빙 오픈소스 프레임워크로, 단일 컨테이너 내에서 여러 모델들을 호스팅하고, 모델을 컨터이너에 동적으로 로드 및 언로드하고 멀티모델 엔드포인트에 필요한 http 프론트엔드 및 모델 관리 기능을 제공합니다. 자세한 내용은 아래 웹페이지를 참조해 주세요.
- https://github.com/awslabs/multi-model-server

### SageMaker Inference Toolkit
SageMaker 상에서 MMS를 좀 더 쉽고 편하게 배포할 수 있는 high-level 어플리케이션으로 배포한 툴킷으로 멀티모델 엔드포인트를 쉽게 배포할 수 있는 설정 및 인터페이스를 지원합니다. 자세한 내용은 아래 웹페이지를 참조해 주세요.
- https://github.com/aws/sagemaker-inference-toolkit

<br>

## 1. Build Your Own Container
---
사용자 지정 SageMaker 컨테이너를 로컬에서 빌드하고 Amazon ECR(Elastic Container Registry)에 등록합니다. ECR은 컨테이너 이미지를 손쉽게 저장, 관리 및 배포할 수 있는 완전 관리형 도커 컨테이너 레포지토리입니다.

### 1.1. Import libraries

In [1]:
%load_ext autoreload
%autoreload 2
!pip install jsonlines

In [2]:
import boto3
import jsonlines
import json
import time

from sagemaker import get_execution_role
from time import gmtime, strftime

### 1.2. Define model handler

`container/model_handler.py` 파일은 GluonTS로 훈련한 모델을 추론하는 사용자 정의 핸들러 클래스입니다.

모델이 메모리에 로드될 때 `initialize` 메소드가 호출됩니다. 이 예제에서는 model_dir의 모델 아티팩트를 GluonTS Predictor 클래스로 로드합니다.

모델을 호출할 때 `handle` 메소드가 호출됩니다. 이 예시에서는 입력 페이로드의 유효성을 검사한 다음 입력값을 GluonTS Predictor 클래스로 전달하여 출력을 반환합니다. 이 핸들러 클래스는 컨테이너에 로드된 모든 모델에 대해 인스턴스화되므로, 핸들러의 상태는 모델간에 공유되지 않습니다.

In [3]:
%%writefile container/model_handler.py
"""
ModelHandler defines an example model handler for load and inference requests for MXNet CPU models
"""
from collections import namedtuple
import glob
import json
import logging
import io
import os
import re

import mxnet as mx
import numpy as np
import sys

from pathlib import Path
from gluonts.model.predictor import Predictor
from gluonts.dataset.common import ListDataset

class ModelHandler(object):
    """
    A sample Model handler implementation.
    """

    def __init__(self):
        self.initialized = False
        self.mx_model = None
        self.shapes = None
    
    def load_model(self, model_path):
        try:
            predictor = Predictor.deserialize(Path(model_path))
            print('Model loaded from %s'%model_path)
        except:
            print('Unable to load the model %s'%model_path)
            sys.exit(1)
        return predictor

    def initialize(self, context):
        """
        Initialize model. This will be called during model loading time
        :param context: Initial context contains model server system properties.
        :return:
        """
        self.initialized = True
        properties = context.system_properties
        # Contains the url parameter passed to the load request
        model_dir = properties.get("model_dir") 
        gpu_id = properties.get("gpu_id")

        # Load Gluonts Model
        self.mx_model = self.load_model(model_dir)

    def preprocess(self, request):
        """
        Transform raw input into model input data.
        :param request: list of raw requests
        :return: list of preprocessed model input data
        """
        # Take the input data and pre-process it make it inference ready

        json_list = []
        # for each request
        for idx, data in enumerate(request):
            # Read the bytearray of the jsonline from the input
            jsonline_arr = data.get('body')  
            # Input json is in bytearray, convert it to string
            jsonline_str = jsonline_arr.decode("utf-8")
            # split the json lines
            json_list_request = []
            # for each time series
            for line in io.StringIO(jsonline_str):
                json_record = json.loads(line)
                json_list_request.append(json_record)
            json_list.append(json_list_request)
        return json_list

    def inference(self, model_input):
        """
        Internal inference methods
        :param model_input: transformed model input data list
        :return: list of inference output in NDArray
        """
        forecast_list = []
        for model_input_request in model_input:
            forecast = list(self.mx_model.predict(ListDataset(
                      model_input_request,
                      freq = self.mx_model.freq
            )))
            forecast_list.append(forecast)
        return forecast_list

    def postprocess(self, inference_output):
        """
        Return predict result in as list.
        :param inference_output: list of inference output
        :return: list of predict results
        """
        ret = []
        quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        # for each request
        for inference_output_request in inference_output:
            ret_request = []
            # for each time series
            for i in inference_output_request:
                l = {}
                l["item_id"] = i.item_id
                l["quantiles"] = {}
                for q in quantiles:
                    l["quantiles"][str(q)] = i.quantile(q).tolist()
                l["mean"] = i.mean.tolist()
                ret_request.append(json.dumps(l))
            ret.append('\n'.join(ret_request) + '\n')
        return ret
        
    def handle(self, data, context):
        """
        Call preprocess, inference and post-process functions
        :param data: input data
        :param context: mms context
        """
        
        model_input = self.preprocess(data)
        model_out = self.inference(model_input)
        return self.postprocess(model_out)

_service = ModelHandler()


def handle(data, context):
    if not _service.initialized:
        _service.initialize(context)

    if data is None:
        return None

    return _service.handle(data, context)

Overwriting container/model_handler.py


### 1.3. Unit testing for the model handler

사용자 정의 도커 컨테이너를 빌드하기 전에 아래와 같이 유닛 테스트(__`container/test_model_handler.py`__) 를 수행하는 것을 권장합니다.

In [4]:
%%bash

cd container
pytest -v test_model_handler.py

============================= test session starts ==============================
platform linux -- Python 3.6.13, pytest-6.2.2, py-1.10.0, pluggy-0.13.1 -- /home/ec2-user/anaconda3/envs/mxnet_p36/bin/python
cachedir: .pytest_cache
rootdir: /home/ec2-user/SageMaker/time-series-on-aws-hol/electricity-demand-byoc-multimodel-endpoint/container
plugins: anyio-2.1.0
collecting ... collected 5 items

test_model_handler.py::test_load_model PASSED                            [ 20%]
test_model_handler.py::test_initialize PASSED                            [ 40%]
test_model_handler.py::test_preprocess PASSED                            [ 60%]
test_model_handler.py::test_handle[5-quantiles0] PASSED                  [ 80%]
test_model_handler.py::test_handle[25-quantiles1] PASSED                 [100%]

============================== 5 passed in 5.46s ===============================


### 1.4. Define Docker Entrypoint

MMS(Multi Model Server)는 프레임워크에 구애받지 않도록 설계되었기 때문에, 모든 프레임워크의 백엔드 엔진 역할을 할 수 있는 충분한 유연성을 제공합니다.

SageMaker Inference Toolkit은 SageMaker 멀티 모델 엔드포인트와 호환되는 방식으로 MMS를 부트스트랩하는 라이브러리이며 모델 당 worker 수(number of workers per model)와 같은 중요한 파라메터를 조정할 수 있습니다. 

이 예제의 추론 컨테이너는 Inference Toolkit을 사용하여 __`container/dockerd-entrypoint.py`__ 파일의 main()함수에서 MMS를 시작합니다.

In [5]:
%%writefile container/dockerd-entrypoint.py

import subprocess
import sys
import shlex
import os
from retrying import retry
from subprocess import CalledProcessError
from sagemaker_inference import model_server

def _retry_if_error(exception):
    return isinstance(exception, CalledProcessError or OSError)

@retry(stop_max_delay=1000 * 50,
       retry_on_exception=_retry_if_error)
def _start_mms():
    # by default the number of workers per model is 1, but we can configure it through the
    # environment variable below if desired.
    # os.environ['SAGEMAKER_MODEL_SERVER_WORKERS'] = '2'
    model_server.start_model_server(handler_service='/home/model-server/model_handler.py:handle')

def main():
    if sys.argv[1] == 'serve':
        _start_mms()
    else:
        subprocess.check_call(shlex.split(' '.join(sys.argv[1:])))

    # prevent docker exit
    subprocess.call(['tail', '-f', '/dev/null'])
    
main()

Overwriting container/dockerd-entrypoint.py


### 1.5. Building and registering a container

아래의 셸 스크립트는 먼저 MMS를 프런트엔드로 사용하는 사용자 정의 도커 이미지를 빌드합니다. 그런 다음 사용자 계정의 ECR 리포지토리에 도커 이미지를 업로드합니다. 최초로 도커 이미지 빌드 시에는 수행 시간이 오래 소요됩니다.

#### Tip
아래 코드 셀에서 No space left 관련 오류가 발생하면, 도커 이미지/컨테이너가 저장될 폴더를 SageMaker 볼륨으로 변경해 주세요.

`$ mkdir -p /home/ec2-user/SageMaker/docker_dir`
`$ sudo vim /etc/sysconfig/docker`

`/etc/sysconfig/docker` 파일을 아래 내용으로 수정합니다.

`OPTIONS="--default-ulimit nofile=1024:4096 -g /home/ec2-user/SageMaker/docker_dir"`

아래 커맨드로 도커를 재시작 후, 폴더가 정상적으로 변경되었는지 확인합니다.

`$ sudo service docker restart`
`% sudo docker info | grep Root`

In [16]:
%%bash

# The name of our algorithm
algorithm_name=demo-sagemaker-multimodel-gluonts

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -q -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
sha256:8a5bd537b2381b4cc138292f8c5f8b36b5af568bd8ac38ca6096feaf7306d765
The push refers to repository [143656149352.dkr.ecr.us-east-1.amazonaws.com/demo-sagemaker-multimodel-gluonts]
de603ec13ff6: Preparing
d1a8893f3396: Preparing
37fa9d8880f0: Preparing
6fc2d732ad49: Preparing
d52da9f1bb83: Preparing
139386551f5a: Preparing
40edb07decfe: Preparing
4581ecc3dbe5: Preparing
97eb5c7d2a69: Preparing
fe8c9427beab: Preparing
c56acba6bc0f: Preparing
6f1a2ed3991b: Preparing
6f15325cc380: Preparing
1e77dd81f9fa: Preparing
030309cad0ba: Preparing
6f1a2ed3991b: Waiting
97eb5c7d2a69: Waiting
6f15325cc380: Waiting
fe8c9427beab: Waiting
c56acba6bc0f: Waiting
1e77dd81f9fa: Waiting
030309cad0ba: Waiting
4581ecc3dbe5: Waiting
40edb07decfe: Waiting
139386551f5a: Waiting
de603ec13ff6: Pushed
d1a8893f3396: Pushed
37fa9d8880f0: Pushed
d52da9f1bb83: Pushed
97eb5c7d2a69: Layer already exists
fe8c9427beab: Layer already exists
c56acba6bc0f: Layer already exists
6f1a2ed3991b: Layer already exis

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



<br>

## 2. Deploy Models as Sagemaker Multi-Model Endpoint and Invoke the Endpoint
---

컨테이너를 ECR에 등록한 후, 모델을 SageMaker 멀티모델 엔드포인트로 배포하고 엔드포인트를 호출합니다.

### 2.1. Set up the environment

먼저, 멀티모델 엔드포인트에서 호출할 모델 아티팩트의 S3 버킷과 prefix를 정의해야 합니다. 또한, 모델 아티팩트 및 ECR 이미지에 대한 액세스 권한을 부여할 IAM role을 정의해야 합니다.

In [24]:
sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

bucket = 'sagemaker-{}-{}'.format(region, account_id)
prefix = 'demo-multimodel-gluonts-endpoint'

role = get_execution_role()

models_dir = "models"

### 2.2. Create a multi-model endpoint
#### Import models into hosting

다중 모델 엔드 포인트에 대한 모델 엔터티를 생성할 때 컨테이너의 `ModelDataUrl`은 엔드포인트에서 호출할 수 있는 모델 아티팩트가 있는 S3 prefix입니다. 나머지 S3 경로는 모델을 호출할 때 지정됩니다.

또한, 컨테이너가 여러 모델들을 호스팅 함을 나타내기 위해 `'Mode': 'MultiModel'`로 지정합니다.

In [25]:
model_name = 'DEMO-MultiModelGluonTSModel' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://s3-{}.amazonaws.com/{}/{}/{}/'.format(region, bucket, prefix, models_dir)
container = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account_id, region, 'demo-sagemaker-multimodel-gluonts')

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container,
    'ModelDataUrl': model_url,
    'Mode': 'MultiModel'
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: DEMO-MultiModelGluonTSModel2021-04-08-14-47-12
Model data Url: https://s3-us-east-1.amazonaws.com/sagemaker-us-east-1-143656149352/demo-multimodel-gluonts-endpoint/models/
Container image: 143656149352.dkr.ecr.us-east-1.amazonaws.com/demo-sagemaker-multimodel-gluonts:latest
Model Arn: arn:aws:sagemaker:us-east-1:143656149352:model/demo-multimodelgluontsmodel2021-04-08-14-47-12


#### Create endpoint configuration

엔드포인트 설정 생성은 단일 모델 엔드포인트와 동일한 방식입니다.

In [26]:
endpoint_config_name = 'DEMO-MultiModelGluonTSEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.m5.xlarge',
        'InitialInstanceCount': 2,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: DEMO-MultiModelGluonTSEndpointConfig-2021-04-08-14-47-12
Endpoint config Arn: arn:aws:sagemaker:us-east-1:143656149352:endpoint-config/demo-multimodelgluontsendpointconfig-2021-04-08-14-47-12


#### Create the multi model endpoint

엔드포인트 생성 또한 단일 모델 엔드포인트와 동일한 방식입니다.

In [27]:
endpoint_name = 'DEMO-MultiModelGluonTSEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: DEMO-MultiModelGluonTSEndpoint-2021-04-08-14-47-12
Endpoint Arn: arn:aws:sagemaker:us-east-1:143656149352:endpoint/demo-multimodelgluontsendpoint-2021-04-08-14-47-12
Endpoint Status: Creating
Waiting for DEMO-MultiModelGluonTSEndpoint-2021-04-08-14-47-12 endpoint to be in service...


### 2.3. Invoke models

호스팅 엔드포인트가 생성되었으면, 이전 과정에서 S3에 업로드한 모델을 호출합니다. 최초 호출 시에 SageMaker가 S3에서 인스턴스로 모델 아티팩트를 다운로드하고 컨테이너에 로드하기 때문에 호출 속도가 느릴 수 있습니다.

#### Invoke the Mean Model

먼저 모델을 호출하기 위한 페이로드로 두 개의 시계열을 준비한 다음 boto3 API의 `InvokeEndpoint`를 호출합니다. `TargetModel` 필드는 모델을 생성 할 때 `ModelDataUrl`에 지정된 S3 prefix와 연결되어 S3에서 모델의 위치를 생성합니다.

In [28]:
def read_data(file_path):
    data = []
    with jsonlines.open(file_path) as reader:
        for obj in reader:
            data.append(obj)
    return data

payload_jsonline = read_data('data/test.json')

In [29]:
n_time_series = 2 # select 2 time series for quick response
payload_list = []
for p in payload_jsonline[:n_time_series]:
    payload_list.append(json.dumps(p))
payload = '\n'.join(payload_list)

In [30]:
def get_inference_result(runtime_sm_client, endpoint_name, model_filename, request_body):

    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        TargetModel=model_filename, # this is the rest of the S3 path where the model artifacts are located
        Body=payload)
    result = response['Body'].read().decode('utf-8')
    
    return result 

In [31]:
%%time
result_mean = get_inference_result(runtime_sm_client, endpoint_name, 'MeanPredictor.tar.gz', payload)
print(result_mean, sep='\n')

{"item_id": "0", "quantiles": {"0.1": [1.810399608829874, 1.8131818477080341, 1.4484505582328373, 1.5365462275284603, 1.6998268218274881, 1.6555417070853777, 1.5886599334362295, 1.639505339658033, 1.6866150796250512, 1.5103819152027935, 1.6573544609794224, 1.477236348409322], "0.2": [2.0260910161621637, 1.913265705112409, 2.021964203687525, 2.0387477820909647, 1.864360129659764, 1.9669942837326502, 1.8864997919735464, 1.9759512325561546, 2.0050252881844894, 1.8281849066256193, 1.9122738252257485, 1.763851960399548], "0.3": [2.2130347358771334, 2.1675520943536593, 2.1846103985492267, 2.1434094349881727, 2.1221213832251533, 2.185128123979893, 2.091302927070566, 2.2356458614508394, 2.1526344102236483, 2.057776659342696, 2.113431188791634, 2.101374557166642], "0.4": [2.458741711111396, 2.3912627576654364, 2.291338040845962, 2.341335484458328, 2.315010654809142, 2.379071081558289, 2.328475072132477, 2.425226805988193, 2.419037162644797, 2.271509438164376, 2.4003791674309682, 2.3050799167047

최초 호출 이후, 두 번째로 동일한 모델을 호출하면 이미 인스턴스에 다운로드되고 컨테이너에 로드되므로 추론이 빠르게 수행됩니다.

In [32]:
%%time
result_mean = get_inference_result(runtime_sm_client, endpoint_name, 'MeanPredictor.tar.gz', payload)
print(result_mean, sep='\n')

{"item_id": "0", "quantiles": {"0.1": [1.4654508260529162, 1.423891781689255, 1.7757414822262154, 1.5864920155198812, 1.5900224350424936, 1.6646010649063432, 1.5181594989210836, 1.8364850132854333, 1.4626581356351, 1.470558835612606, 1.475199890108488, 1.4323098654172937], "0.2": [1.7768910447676578, 1.839694094921167, 2.146634543612159, 1.903434177590697, 1.9059980577206503, 1.8943988644221235, 1.9303880187593374, 2.1528906218255472, 1.9011312788981112, 1.9319513250012816, 1.8426904338757735, 1.8479433735445698], "0.3": [1.9592398504988617, 2.1374060116459663, 2.4041354155496837, 2.092304273981136, 2.090190529897593, 2.2635209121125155, 2.1114415033457847, 2.452898443884457, 2.300691587370901, 2.0984121160878244, 2.2409590866158497, 2.076098552490381], "0.4": [2.217468917503038, 2.3136295758431977, 2.557049906375984, 2.2318131010962796, 2.344065100871651, 2.4444397200257795, 2.3628398849195316, 2.542455693954407, 2.4748433616432886, 2.3036864768588834, 2.3729759310392065, 2.4240059723

#### Invoke multi models

멀티모델 엔드포인트의 힘을 발휘하여 다른 모델 (예: `DeepAREstimator.tar.gz`)을 `TargetModel`로 지정하고 동일한 엔드포인트를 사용하여 추론을 수행할 수 있습니다.

In [33]:
result_naive = get_inference_result(runtime_sm_client, endpoint_name, 'SeasonalNaivePredictor.tar.gz', payload)
result_prophet = get_inference_result(runtime_sm_client, endpoint_name, 'ProphetPredictor.tar.gz', payload)
result_ffn = get_inference_result(runtime_sm_client, endpoint_name, 'SimpleFeedForwardEstimator.tar.gz', payload)
result_deepar = get_inference_result(runtime_sm_client, endpoint_name, 'DeepAREstimator.tar.gz', payload)

In [34]:
print(result_deepar, sep='\n')

{"item_id": "0", "quantiles": {"0.1": [2.5666637420654297, 2.8132734298706055, 2.6296422481536865, 2.8897743225097656, 3.470712423324585, 3.577692985534668, 3.3943748474121094, 2.4244911670684814, 2.2983062267303467, 2.5657153129577637, 2.6624789237976074, 2.5903286933898926], "0.2": [2.7564260959625244, 2.984457015991211, 2.9986135959625244, 3.050806760787964, 3.607384204864502, 3.687776803970337, 3.5164635181427, 2.585606813430786, 2.5066545009613037, 2.725512742996216, 2.7753734588623047, 2.732186794281006], "0.3": [2.8530142307281494, 3.0957539081573486, 3.0873405933380127, 3.184116840362549, 3.8131134510040283, 3.825244426727295, 3.601621150970459, 2.6971001625061035, 2.586515188217163, 2.8266780376434326, 2.87007474899292, 2.8411788940429688], "0.4": [2.9325153827667236, 3.1580684185028076, 3.1143741607666016, 3.3106939792633057, 3.866978883743286, 3.9082369804382324, 3.7005550861358643, 2.7283027172088623, 2.670076847076416, 2.948002576828003, 2.925839900970459, 2.88446307182312

<br>

## 3. Batch Transform in the Multi-Model Server

MMS는 배치 변환(batch transform)을 직접 지원하지 않기 때문에, SageMaker에서 별도로 모델을 생성하고 각 모델에 대해 하나씩 배치 변환을 수행해야 합니다. 아래는 하나의 모델에 대해 배치 변환을 수행하는 예시를 보여줍니다.

### 3.1. Create the Sagemaker model from the model artifact

In [35]:
from time import gmtime, strftime

model = 'DeepAREstimator'
model_name_bt = 'DEMO-GluonTSModel-{}-'.format(model) + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://s3-{}.amazonaws.com/{}/{}/{}/{}.tar.gz'.format(region, bucket, prefix, models_dir, model)
container = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account_id, region, 'demo-sagemaker-multimodel-gluonts')

print('Model name: ' + model_name_bt)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container,
    'ModelDataUrl': model_url,
    'Mode': 'SingleModel'
}

create_model_response = sm_client.create_model(
    ModelName = model_name_bt,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: DEMO-GluonTSModel-DeepAREstimator-2021-04-08-14-56-02
Model data Url: https://s3-us-east-1.amazonaws.com/sagemaker-us-east-1-143656149352/demo-multimodel-gluonts-endpoint/models/DeepAREstimator.tar.gz
Container image: 143656149352.dkr.ecr.us-east-1.amazonaws.com/demo-sagemaker-multimodel-gluonts:latest
Model Arn: arn:aws:sagemaker:us-east-1:143656149352:model/demo-gluontsmodel-deeparestimator-2021-04-08-14-56-02


### 3.2. Start the Batch Transform Job Using the Model Created above

In [36]:
test_data_s3_path = "s3://{}/{}/data/test.json".format(bucket, prefix)
transform_job_name = 'DEMO-GluonTS-{}-BT-'.format(model) + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

transform_input = {
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': test_data_s3_path
            }
        },
        'ContentType': 'application/json',
        'CompressionType': 'None',
        'SplitType': 'Line'
    }

transform_output = {
        'S3OutputPath': 's3://{}/{}/inference-results/{}'.format(bucket,prefix, model),
    }

transform_resources = {
        'InstanceType': 'ml.m5.xlarge',
        'InstanceCount': 1
    }

sm_client.create_transform_job(TransformJobName = transform_job_name,
                        ModelName = model_name_bt,
                        BatchStrategy='SingleRecord',
                        TransformInput = transform_input,
                        TransformOutput = transform_output,
                        TransformResources = transform_resources
)

{'TransformJobArn': 'arn:aws:sagemaker:us-east-1:143656149352:transform-job/demo-gluonts-deeparestimator-bt-2021-04-08-14-56-02',
 'ResponseMetadata': {'RequestId': 'b8aae7a2-1cb4-495c-b4a7-ed16ef62acf2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b8aae7a2-1cb4-495c-b4a7-ed16ef62acf2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '128',
   'date': 'Thu, 08 Apr 2021 14:56:02 GMT'},
  'RetryAttempts': 0}}

### 3.3. Check the Batch Transform Job Status

모든 데이터에 대해 추론을 수행하므로, 약간의 시간이 소요됩니다.

In [37]:
print ('JobStatus')
print('----------')
from time import sleep

describe_response = sm_client.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm_client.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

JobStatus
----------
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed


### 3.4. Inspect Batch Transform Results

In [38]:
s3_client = boto3.client('s3')
s3_client.download_file(Filename='data/test.json.out',
                        Bucket=bucket,
                        Key='{}/inference-results/{}/test.json.out'.format(prefix, model))
test_out_jsonline = read_data('data/test.json.out')
print(test_out_jsonline[:2])

[{'item_id': '0', 'quantiles': {'0.1': [2.5666637420654297, 2.8132734298706055, 2.6296422481536865, 2.8897743225097656, 3.470712423324585, 3.577692985534668, 3.3943748474121094, 2.4244911670684814, 2.2983062267303467, 2.5657153129577637, 2.6624789237976074, 2.5903286933898926], '0.2': [2.7564260959625244, 2.984457015991211, 2.9986135959625244, 3.050806760787964, 3.607384204864502, 3.687776803970337, 3.5164635181427, 2.585606813430786, 2.5066545009613037, 2.725512742996216, 2.7753734588623047, 2.732186794281006], '0.3': [2.8530142307281494, 3.0957539081573486, 3.0873405933380127, 3.184116840362549, 3.8131134510040283, 3.825244426727295, 3.601621150970459, 2.6971001625061035, 2.586515188217163, 2.8266780376434326, 2.87007474899292, 2.8411788940429688], '0.4': [2.9325153827667236, 3.1580684185028076, 3.1143741607666016, 3.3106939792633057, 3.866978883743286, 3.9082369804382324, 3.7005550861358643, 2.7283027172088623, 2.670076847076416, 2.948002576828003, 2.925839900970459, 2.8844630718231

## 4. (Optional) Clean-up the resources

불필요한 과금을 막기 위해 핸즈온을 모두 완료하였으면, 엔드포인트를 삭제합니다.

In [39]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)
sm_client.delete_model(ModelName=model_name_bt)

{'ResponseMetadata': {'RequestId': '2180b4c2-5a2f-48b7-a44b-fc69e69c2e8b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2180b4c2-5a2f-48b7-a44b-fc69e69c2e8b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 08 Apr 2021 15:05:00 GMT'},
  'RetryAttempts': 2}}